# 00 FRED API and Caching

Fetch indicators from FRED and cache raw JSON.


## Table of Contents
- Choose series
- Fetch metadata
- Fetch + cache observations
- Fallback to sample


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Matching Guide
- `docs/guides/01_data/00_fred_api_and_caching.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/01_data/00_fred_api_and_caching.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fetch a few FRED series, cache raw JSON, and convert observations into tidy DataFrames.


## Your Turn: Choose Series


In [ ]:
# TODO: Define a list of FRED series IDs
series_ids = [...]

# Suggested: UNRATE, FEDFUNDS, CPIAUCSL, INDPRO, RSAFS, T10Y2Y



## Your Turn: Fetch Metadata


In [ ]:
from src import fred_api

# TODO: Pick one series and fetch metadata with fred_api.fetch_series_meta
# Print title, units, frequency, seasonal_adjustment
...


## Your Turn: Fetch + Cache Observations


In [ ]:
from src import data as data_utils

# TODO: For each series_id, cache JSON under data/raw/fred/<id>.json
# Hint: data_utils.load_or_fetch_json + fred_api.fetch_series_observations
...


## Fallback


In [ ]:
# TODO: If you do not have FRED_API_KEY set, load data/sample/panel_monthly_sample.csv
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: (see notebook front matter)
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Choose series</summary>

```python
series_ids = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']
series_ids
```

</details>

<details><summary>Solution: Fetch metadata</summary>

```python
from src import fred_api

meta = fred_api.fetch_series_meta('UNRATE')
meta
```

</details>

<details><summary>Solution: Fetch + cache observations</summary>

```python
from src import data as data_utils
from src import fred_api

raw_dir = RAW_DIR / 'fred'
raw_dir.mkdir(parents=True, exist_ok=True)

frames = []
for sid in series_ids:
    payload = data_utils.load_or_fetch_json(
        raw_dir / f'{sid}.json',
        lambda sid=sid: fred_api.fetch_series_observations(sid, start_date='1980-01-01', end_date=None),
    )
    frames.append(fred_api.observations_to_frame(payload, sid))

panel = pd.concat(frames, axis=1).sort_index()
panel.head()
```

</details>

<details><summary>Solution: Fallback to sample</summary>

```python
import pandas as pd

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
panel.head()
```

</details>

